In [1]:
# Set up the environment
import time
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.common.exceptions import NoSuchElementException

In [2]:
driver = webdriver.Chrome()

In [3]:
# create pagination 
 
# driver wait 10 seconds until the page loaded 
driver.implicitly_wait(10) 
 

driver.get('https://allears.net/dining/menu/search/all/all/all/quick-service/') 

In [4]:
restaurant_info = {'Title':[],'Urls':[]}

# find each restaurant card
titles = driver.find_elements("xpath", '//div[@class="dining-card-slide item card-slide"]/a')
links = driver.find_elements("xpath", '//div[@class="dining-card-slide item card-slide"]/a')

for title,link in zip(titles,links): 
    restaurant_info['Title'].append(title.get_attribute('textContent').strip())
    restaurant_info['Urls'].append(link.get_attribute('href'))

import pandas as pd  

restaurant_df = pd.DataFrame(restaurant_info)
restaurant_df

,Title,Urls
0,Tortuga Tavern - Lunch/Dinner,https://allears.net/dining/menu/tortuga-tavern...
1,Friar's Nook - Breakfast,https://allears.net/dining/menu/friars-nook/br...
2,Friar's Nook - Lunch/Dinner,https://allears.net/dining/menu/friars-nook/lu...
3,Gaston's Tavern - All-Day,https://allears.net/dining/menu/gastons-tavern...
4,Pinocchio Village Haus - Lunch/Dinner,https://allears.net/dining/menu/pinocchios-vil...
...,...,...
133,Leaning Palms - Lunch,https://allears.net/dining/menu/leaning-palms/...
134,Lowtide Lou's - Lunch,https://allears.net/dining/menu/lowtide-lous/l...
135,Snack Shack - Lunch,https://allears.net/dining/menu/snack-shack/lu...
136,Surf Doggies - Snacks,https://allears.net/dining/menu/surf-doggies/s...


In [11]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
import pandas as pd
import time

Menu_info = {'Restaurant': [], 'Item_name': [], 'Description': [], 'Price': [], 'Dining_plan': []}

options = Options()
# options.add_argument("--headless")  # keep headless off for debugging
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")

driver = webdriver.Chrome(options=options)

for index, link in enumerate(restaurant_df['Urls']):
    restaurant = restaurant_df['Title'].iloc[index]

    time.sleep(5)

    if index % 10 == 0 and index != 0:
        driver.quit()
        driver = webdriver.Chrome(options=options)

    try:
        driver.set_page_load_timeout(75)
        driver.get(link)
    except TimeoutException:
        print(f"Timeout loading {link}")
        continue

    driver.implicitly_wait(5)

    menu_items = driver.find_elements("xpath", '//div[@class="menuItems__item item card-slide flex"]')

    for menu_item in menu_items:
        try:
            dining_plan = menu_item.find_element("xpath", './/a[@title="Disney Dining Plan (DDP) Snack. Items marked with   this symbol can be purchased with one DDP snack entitlement. Some   examples of snacks: frozen ice cream novelty, popsicle, fruit bar;   popcorn single serving box (scoop); single piece of whole fruit; single   serving bag of snacks; 20 oz. bottled drink (Coke, Diet Coke or Sprite)   or water; 22 oz. fountain soft drink; 12 oz. coffee, hot chocolate or   tea; or single serving prepackaged milk or juice. To learn more about the Disney Dining Plans visit: https://allears.net/pl/packages.htm"]').get_attribute('title')
            if not dining_plan:
                continue
        except:
            continue

        try:
            name = menu_item.find_element("xpath", './/span[1]').get_attribute("textContent").strip()
        except:
            name = None

        try:
            description = menu_item.find_element("xpath", './/span[2]').get_attribute("textContent").strip()
        except:
            description = None

        try:
            price = menu_item.find_element("xpath", './/span[3]').get_attribute("textContent").strip()
        except:
            price = None

        Menu_info['Restaurant'].append(restaurant)
        Menu_info['Item_name'].append(name)
        Menu_info['Description'].append(description)
        Menu_info['Price'].append(price)
        Menu_info['Dining_plan'].append(dining_plan)

    if index % 20 == 0 and index != 0:
        pd.DataFrame(Menu_info).to_csv('menu_progress.csv', index=False)
        print(f"Saved progress at index {index}")

driver.quit()


Timeout loading https://allears.net/dining/menu/golden-oak-outpost/lunch-dinner/
Timeout loading https://allears.net/dining/menu/regal-eagle-smokehouse/lunch-dinner/
Saved progress at index 20
Timeout loading https://allears.net/dining/menu/refreshment-port/lunch-dinner/
Timeout loading https://allears.net/dining/menu/creature-comforts-starbucks/all-day/
Saved progress at index 40
Saved progress at index 60
Saved progress at index 80
Saved progress at index 100
Saved progress at index 120


In [76]:
Menu_df = pd.DataFrame(Menu_info)
Menu_df['Dining_plan'] = Menu_df['Dining_plan'].apply(lambda x: 'Dining Plan' if x is not None else None)
Menu_df

,Restaurant,Item_name,Description,Price,Dining_plan
0,Tortuga Tavern - Lunch/Dinner,Fountain Beverages,"Coke, Diet Coke, Sprite, Powerade, Light Lemon...",$4.49,Dining Plan
1,Tortuga Tavern - Lunch/Dinner,Bottled Water,,$3.75,Dining Plan
2,Tortuga Tavern - Lunch/Dinner,"Desert Pear Lemonade Pirate Slushy,",Lemonade,$6.49,Dining Plan
3,Friar's Nook - Breakfast,2024 TIANAS BAYOU- Frozen Bayou Lavender Lemonade,premium lemonade with flavors of lavender insp...,$6.29,Dining Plan
4,Friar's Nook - Breakfast,Chocolate Milk,,$4.49,Dining Plan
5,Friar's Nook - Breakfast,Hot Tea,,$4.29,Dining Plan
6,Friar's Nook - Breakfast,Hot Cocoa,,$4.29,Dining Plan
7,Friar's Nook - Breakfast,Coffee,,$4.29,Dining Plan
8,Friar's Nook - Breakfast,Cold Brew French Roast,,$4.49,Dining Plan
9,Friar's Nook - Breakfast,Strawberry Lemonade Slush,,$6.49,Dining Plan


In [12]:
Menu_df = pd.DataFrame(Menu_info)
Menu_df

,Restaurant,Item_name,Description,Price,Dining_plan
0,Tortuga Tavern - Lunch/Dinner,Fountain Beverages,"Coke, Diet Coke, Sprite, Powerade, Light Lemon...",$4.49,Disney Dining Plan (DDP) Snack. Items marked w...
1,Tortuga Tavern - Lunch/Dinner,Bottled Water,,$3.75,Disney Dining Plan (DDP) Snack. Items marked w...
2,Tortuga Tavern - Lunch/Dinner,"Desert Pear Lemonade Pirate Slushy,",Lemonade,$6.49,Disney Dining Plan (DDP) Snack. Items marked w...
3,Friar's Nook - Breakfast,2024 TIANAS BAYOU- Frozen Bayou Lavender Lemonade,premium lemonade with flavors of lavender insp...,$6.29,Disney Dining Plan (DDP) Snack. Items marked w...
4,Friar's Nook - Breakfast,Chocolate Milk,,$4.49,Disney Dining Plan (DDP) Snack. Items marked w...
...,...,...,...,...,...
2674,Typhoon Tilly's - Lunch,Hot Coffee,regular or decaf,$3.29,Disney Dining Plan (DDP) Snack. Items marked w...
2675,Typhoon Tilly's - Lunch,Frozen Lemonade,blue raspberry,$5.99,Disney Dining Plan (DDP) Snack. Items marked w...
2676,Typhoon Tilly's - Lunch,Hot Cocoa,,$3.29,Disney Dining Plan (DDP) Snack. Items marked w...
2677,Typhoon Tilly's - Lunch,Hot Tea,,$3.29,Disney Dining Plan (DDP) Snack. Items marked w...


In [16]:
Menu_df.to_csv('Menu_Items.csv')

In [14]:
# grab extra menus that failed

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
import pandas as pd
import time

Menu_info1 = Menu_info

options = Options()
# options.add_argument("--headless")  # keep headless off for debugging
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")

driver = webdriver.Chrome(options=options)

names = ["Golden Oak Output - Lunch/Dinner", "Regal Eagle Smokehouse", "Refreshment Port", "Creature Comforts"]
links = ["https://allears.net/dining/menu/golden-oak-outpost/lunch-dinner/", "https://allears.net/dining/menu/regal-eagle-smokehouse/lunch-dinner/", "https://allears.net/dining/menu/refreshment-port/lunch-dinner/", "https://allears.net/dining/menu/creature-comforts-starbucks/all-day/"]
for name, link in zip(names, links):
    restaurant = name

    time.sleep(5)

    try:
        driver.set_page_load_timeout(115)
        driver.get(link)
    except TimeoutException:
        print(f"Timeout loading {link}")
        continue

    driver.implicitly_wait(5)

    menu_items = driver.find_elements("xpath", '//div[@class="menuItems__item item card-slide flex"]')

    for menu_item in menu_items:
        try:
            dining_plan = menu_item.find_element("xpath", './/a[@title="Disney Dining Plan (DDP) Snack. Items marked with   this symbol can be purchased with one DDP snack entitlement. Some   examples of snacks: frozen ice cream novelty, popsicle, fruit bar;   popcorn single serving box (scoop); single piece of whole fruit; single   serving bag of snacks; 20 oz. bottled drink (Coke, Diet Coke or Sprite)   or water; 22 oz. fountain soft drink; 12 oz. coffee, hot chocolate or   tea; or single serving prepackaged milk or juice. To learn more about the Disney Dining Plans visit: https://allears.net/pl/packages.htm"]').get_attribute('title')
            if not dining_plan:
                continue
        except:
            continue

        try:
            name = menu_item.find_element("xpath", './/span[1]').get_attribute("textContent").strip()
        except:
            name = None

        try:
            description = menu_item.find_element("xpath", './/span[2]').get_attribute("textContent").strip()
        except:
            description = None

        try:
            price = menu_item.find_element("xpath", './/span[3]').get_attribute("textContent").strip()
        except:
            price = None

        Menu_info1['Restaurant'].append(restaurant)
        Menu_info1['Item_name'].append(name)
        Menu_info1['Description'].append(description)
        Menu_info1['Price'].append(price)
        Menu_info1['Dining_plan'].append(dining_plan)

    if index % 20 == 0 and index != 0:
        pd.DataFrame(Menu_info1).to_csv('menu_progress.csv', index=False)
        print(f"Saved progress at index {index}")

driver.quit()


In [15]:
Menu_df = pd.DataFrame(Menu_info1)
Menu_df['Dining_plan'] = Menu_df['Dining_plan'].apply(lambda x: 'Dining Plan' if x is not None else None)
Menu_df

,Restaurant,Item_name,Description,Price,Dining_plan
0,Tortuga Tavern - Lunch/Dinner,Fountain Beverages,"Coke, Diet Coke, Sprite, Powerade, Light Lemon...",$4.49,Dining Plan
1,Tortuga Tavern - Lunch/Dinner,Bottled Water,,$3.75,Dining Plan
2,Tortuga Tavern - Lunch/Dinner,"Desert Pear Lemonade Pirate Slushy,",Lemonade,$6.49,Dining Plan
3,Friar's Nook - Breakfast,2024 TIANAS BAYOU- Frozen Bayou Lavender Lemonade,premium lemonade with flavors of lavender insp...,$6.29,Dining Plan
4,Friar's Nook - Breakfast,Chocolate Milk,,$4.49,Dining Plan
...,...,...,...,...,...
2752,Creature Comforts,"Honey Almondmilk Cold Brew,",,$4.79,Dining Plan
2753,Creature Comforts,Iced Chocolate Almondmilk Shaken Espresso,,$6.49,Dining Plan
2754,Creature Comforts,Iced Brown Sugar Oatmilk Shaken Espresso,,$6.49,Dining Plan
2755,Creature Comforts,"Chocolate Cream Cold Brew,",,$5.79,Dining Plan


In [18]:
restaurant_df.to_csv('Restaurants.csv')